This notebook ???

In [1]:
%load_ext lab_black
%load_ext watermark

import numpy as np
import pandas as pd

from pathlib import Path

In [25]:
basepath = Path("../../../Downloads/db_iris_all/activite_residents/")
files_in_path = basepath.glob("*.xls")

# load and concat files (heavy):
actifs = pd.DataFrame()
for file in files_in_path:
    df = pd.read_excel(
        file,
        header=5,
        sheet_name="IRIS",
        usecols=["DEP", "LIBCOM", f"C{file.stem[-2:]}_ACTOCC1564"],
        nrows=40_500,
    )
    df = df[df.DEP == "75"].reset_index(drop=True).drop("DEP", axis=1)
    actifs = pd.concat([actifs, df], axis=1)

# drop duplicated column values:
actifs = actifs.T.drop_duplicates().T
# drop duplicated column names:
actifs = actifs.loc[:, ~actifs.columns.duplicated()]

# extract district number:
actifs["LIBCOM"] = actifs.LIBCOM.str.extract("(\d+)").astype(int)
actifs = actifs.rename(columns={"LIBCOM": "arrondissement"})

# aggregate by district and prettify columns:
actifs = actifs.groupby("arrondissement").sum()
actifs.columns = actifs.columns.str[1:3].astype(int) + 2000
actifs.columns.name = "actifs occupes"
actifs = actifs.sort_index(axis=1)
actifs

actifs occupes,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
arrondissement,,,,,,,,,,,
1,9485.059228,9546.148694,9469.633224,9665.691628,9558.180760,9453.454849,9110.708956,8883.849949,8438.116225,8135.217082,8061.414072
2,11895.337850,12395.559350,12449.383546,12794.287723,13079.132959,12897.067260,12705.574946,12511.638294,12443.876247,12172.283638,11953.850833
3,18781.798429,18794.248763,19187.221407,19528.009963,19633.785420,19676.408336,19624.664004,19427.440000,19250.600928,19080.631402,18880.704683
4,14717.129875,14474.184108,14170.410759,14383.516785,14349.181220,14077.461530,14083.926773,13852.854439,13655.120630,13745.463101,13794.282329
5,28127.212305,28896.540584,28718.970489,28261.753714,27791.574245,27907.229912,27479.569920,27486.622826,27370.326561,27153.136723,26531.098045
6,19400.728497,19464.044287,18987.594588,18698.452612,18687.445790,18872.413593,18486.423858,18369.356119,17947.978542,17625.289921,16819.451985
7,25210.101682,25555.120417,25470.719099,25634.964674,25939.173209,25646.194524,25270.864522,24984.518029,24645.281422,24005.170224,23397.139726
8,18334.549675,18373.028456,18104.717871,18953.683842,19428.111309,19149.890060,18470.260890,18698.608989,17966.781661,17511.445134,17298.062602
9,30990.448950,31214.296310,31615.515379,32296.599520,32337.003951,32254.221120,32025.363230,32002.779042,32167.496470,32271.121015,32442.704870
